## Project 1, PSI course on Numerical Methods

### Hassan Khalvati

In [1]:
using CSV #Julia package for handling text data
using DataFrames #Julia package for tabular data tools
using WGLMakie  #Julia package for visualizations
using Optim #Julia package for optimization

In [2]:
dataset = CSV.read("data.csv",DataFrame)

Row,row,x,y,sigma_y,sigma_x,rho_xy
,Int64,Int64,Int64,Int64,Int64,Float64
1,1,201,592,61,9,-0.84
2,2,244,401,25,4,0.31
3,3,47,583,38,11,0.64
4,4,287,402,15,7,-0.27
5,5,203,495,21,5,-0.33
6,6,58,173,15,9,0.67
7,7,210,479,27,4,-0.02
8,8,202,504,14,4,-0.05
9,9,198,510,30,11,-0.84


The aim of this project is to find the best fit formula for the given data set.
As the first step, we need to see how the dataset looks like. Based on the data frame above, we see that it also includes the error data. Hence, we can plot the data points, and their error bars:

In [3]:
f = Figure()
Axis(f[1, 1])
errorbars!(dataset.x, dataset.y, dataset.sigma_y)
scatter!(dataset.x,dataset.y)
f

In this section, we are going to fit a quadratic polynomial to out data set.
As we all know, the second orfer polynomial in the general form is 
$$ y = a x^2 + b x + c $$
which during the fitting process, the coefficients of a, b, and c, would be our tuning parameters.
At the same time, I also include the linear fit results for comparison purposes. For the linear, the function is:
$$
y = m x + h
$$


But, first let's plot the function with some given values of a, b, and c. Thhese values, we call them by_eye value which mean it's been estimated by eye, or checking the plot to see which value set would best represent the data. 

## Plot for the actual scattered data, and a linear, and quadratic funciton with an guessing fitting parameters

In [4]:
a_eye = 0.001;
b_eye = 2.0;
c_eye = 50.0;

m_eye = 2.0
h_eye = 50.0
xx = LinRange(40,300, 30)
f = Figure()
Axis(f[1, 1])
errorbars!(dataset.x, dataset.y, dataset.sigma_y)
scatter!(dataset.x,dataset.y)
yy_quad = a_eye.*xx.^2 .+ b_eye.*xx .+ c_eye
yy_lin = m_eye .*xx .+ h_eye
lines!(xx, yy_quad, color= :black )
lines!(xx,yy_lin, color=:grey)
f

As we did in the class, we can remove couple of points from the beginning and the end of the data set. Because they look to be outliers. But we need to be careful about how we are removing the points, because the dataset is not sorted in x values. So these outliers are mainly the first  5 points

In [5]:
data = dataset[5:end,:]

Row,row,x,y,sigma_y,sigma_x,rho_xy
,Int64,Int64,Int64,Int64,Int64,Float64
1,5,203,495,21,5,-0.33
2,6,58,173,15,9,0.67
3,7,210,479,27,4,-0.02
4,8,202,504,14,4,-0.05
5,9,198,510,30,11,-0.84
6,10,158,416,16,7,-0.69
7,11,165,393,14,5,0.3
8,12,201,442,25,5,-0.46
9,13,157,317,52,5,-0.03


If we plot it again we would see that: 

In [6]:
    a_eye = 0.001;
    b_eye = 2;
    c_eye = 50;
    xx = LinRange(40,300, 30)
    f = Figure()
    Axis(f[1, 1])
    errorbars!(data.x, data.y, data.sigma_y)
    scatter!(data.x,data.y)
    yy = a_eye.*xx.^2 .+ b_eye.*xx .+ c_eye
    lines!(xx, yy, color=:black)
    yy_lin = m_eye .*xx .+ h_eye
    lines!(xx,yy_lin, color=:grey)

    f

Now, it looks better, and we can start our fitting process.
The method that we are aimed to use, is the "Chi_Squared" method based on this paper https://arxiv.org/pdf/1008.4686v1.pdf.
In this method we would basically minimize an objective function $\chi^2$, which is:
$$
\chi^2 = \sum_i^N \frac{|y_i - f(x_i)|^2}{\sigma^2_{yi}}
$$
In which, the f is the fitting function, and the y comes from the data.
In this manner, we need to define our objective funciton, which we would like to optimize that. In this case it is the difference between the y data, and the quadratic funciton values, for each inout set of a,b, and c. 

In [11]:
function objective_lin(params, x, y, sigma)
    m = params[1] # input parameters for the funciton which are our 2 tunning coefficients of linear function
    h = params[2]

    f = m .* x .+ h
#     return sum(abs.(y .- f))
    return sum(abs.(y .- f).^2 ./ sigma.^2)

end

objective_lin (generic function with 1 method)

In [12]:
function objective_quad(params, x, y, sigma)
    a = params[1] # input parameters for the funciton which are our 3 tunning coefficients of 2nd order polynomial
    b = params[2]
    c = params[3]
    
    f = a .* x.^2 .+ b.*x .+c
#     return sum(abs.(y .- f))
    return sum(abs.(y .- f).^2 ./ sigma.^2)

end

objective_quad (generic function with 1 method)

In [13]:
quad_starting_points = [a_eye,b_eye,c_eye]
lin_starting_points = [m_eye,h_eye]
lin_result = optimize(linparam -> objective_lin(linparam, data.x, data.y, data.sigma_y),lin_starting_points);
quad_result = optimize(quadparam -> objective_quad(quadparam, data.x, data.y, data.sigma_y),quad_starting_points);

from just simple minimizing our objective, we can get the values below:

In [14]:
a_opt, b_opt, c_opt = Optim.minimizer(quad_result);
m_opt, h_opt = Optim.minimizer(lin_result);

Now, if we plot again and compare all 3 case, the data, the polynomila with eye estimated values and polynomial with optimised values, we would see:

In [15]:
a_eye = 0.001;
b_eye = 2;
c_eye = 50;
xx = LinRange(40,300, 30)
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x,data.y)
yy_quad = a_eye.*xx.^2 .+ b_eye.*xx .+ c_eye
lines!(xx, yy, color=:black)
yy_lin = m_eye .*xx .+ h_eye
lines!(xx,yy_lin, color=:grey)

opt_yy_quad = a_opt.*xx.^2 .+ b_opt.*xx .+ c_opt
lines!(xx, opt_yy_quad,color=:red )

opt_yy_lin = m_opt.*xx .+ h_opt
lines!(xx, opt_yy_lin,color=:blue )


f

From here, I am going to drop the eye estimated values and only compare the best fits from linear and quadratic funtions:

In [16]:
xx = LinRange(40,300, 30)
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x,data.y)

opt_yy_quad = a_opt.*xx.^2 .+ b_opt.*xx .+ c_opt
lines!(xx, opt_yy_quad,color=:red )

opt_yy_lin = m_opt.*xx .+ h_opt
lines!(xx, opt_yy_lin,color=:blue )


f

So far, we have only naively minimized the differnce between the value of quadratic funciton and out dataset, and we got some optimized value for polynomila coefficients. 
But, a more formal way to find the best fit. or optimum values for quadratic coefficients rather than using the $\chi\_squared$ is to use gaussian function as our objective funciton. 
Choosing guassian funciton, means that we are assuming that the data is actually comming from out fitting funciton, but there are some statistical gaussian errors that have been added to each data point. In another word, we are considering any diviation from out exact fitting fuciton to be gaussian. 
The gaussian funciton in general form is 
$$
p = \frac{1}{\sqrt{2 \pi \sigma_{yi}^2}} exp( - \frac{(y_i - f(x_i)^2}{2 \sigma_{yi}^2} ) 
$$
To minimize this funciton, we can simplify our method, to maximize the -log of gaussian funciton, which is equivalence to minimizing the gaussian function itself. 


In [17]:
function Gauss_objective_quad(params, x, y, sigma) # Gaussian objective function for 2nd order polynomial
    a = params[1] # input parameters for the funciton which are our 3 tunning coefficients of 2nd order polynomial
    b = params[2]
    c = params[3]
    
    f = a .* x.^2 .+ b.*x .+c
   return -sum(
        -log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- f).^2 / sigma.^2)
end

Gauss_objective_quad (generic function with 1 method)

In [18]:
function Gauss_objective_lin(params, x, y, sigma) # Gaussian objective function for linear funciton
    m = params[1] # input parameters for the funciton which are our 2 tunning coefficients of linear function
    h = params[2]

    f = m .* x .+ h
   return -sum(
        -log.(sigma * sqrt(2 * π)) .-0.5 .* (y .- f).^2 / sigma.^2)
end

Gauss_objective_lin (generic function with 1 method)

In [19]:
quad_starting_points = [a_eye,b_eye,c_eye]
lin_starting_points = [m_eye,h_eye]
Gauss_quad_result = optimize(Gauss_quadparam -> Gauss_objective_quad(Gauss_quadparam, data.x, data.y, data.sigma_y),quad_starting_points);
Gauss_lin_result = optimize(Gauss_linparam -> Gauss_objective_lin(Gauss_linparam, data.x, data.y, data.sigma_y),lin_starting_points);

In [20]:
a_gauss, b_gauss, c_gauss = Optim.minimizer(Gauss_quad_result);
m_gauss, h_gauss = Optim.minimizer(Gauss_lin_result);

Let's plot again to see the if there is any improvement relative to the $\chi\_squared$

In [21]:
xx = LinRange(40,300, 30)
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x,data.y)

opt_yy_quad = a_opt.*xx.^2 .+ b_opt.*xx .+ c_opt
lines!(xx, opt_yy_quad,color=:black )
opt_yy_lin = m_opt.*xx .+ h_opt
lines!(xx, opt_yy_lin,color=:grey )

gauss_yy_quad = a_gauss.*xx.^2 .+ b_gauss.*xx .+ c_gauss
lines!(xx, gauss_yy_quad, color=:blue)

gauss_yy_lin = m_gauss.*xx .+ h_gauss
lines!(xx, gauss_yy_lin, color=:red)


f

We can see that second order fits in Black (for $\chi\_squared$) and Blue(for Gaussian) look much better than the linear.  

As we have witnessed from the beginning, there are some of the points which are deviating more than others from a liniear fit, or even for the best quadratic fit, we see the same issue for some of the points. These poiints, can affect our fitting process, especially, when we are doing a linear fit to our dataset. One way to remove these bad point, or so called ourliers, is to cut them from our data set as we can see by eye. But, this method might be good only for thos points which are deviating the overal trend in our data by too much. "JackKnife" is a method to determin point in the dataset, which are affecting our final fitting parameters by too much, however, they might not be distinguishable by eye. In this method, what happens is that we calculate the best fit for many times, and each time, we pull out one of the data points, and in the end, we can compare our fitting results to see in which step we got the most accurate values.  
In the following, I am going to implement the jackknife method on our data set. 

In [22]:
ndata = size(data,1)
# because we are going to have many runs, and each time removing one data point, it means that for 16 data points (npoints), we would get 16 different values for our fitting paramaters
A_jack = zeros(ndata) # quadratic fitting parameters 
B_jack = zeros(ndata)  # quadratic fitting parameters
C_jack = zeros(ndata)  # quadratic fitting parameters
M_jack = zeros(ndata)  # linear fitting parameters
H_jack = zeros(ndata)  # linear fitting parameters

#loop over the pulling points out cases 
for i in 1:ndata
    
    xcopy = copy(data.x)
    deleteat!(xcopy, i)

    ycopy = copy(data.y)
    deleteat!(ycopy, i)

    scopy = copy(data.sigma_y)
    deleteat!(scopy, i)

    
    quad_starting_points = [a_eye,b_eye,c_eye]
    lin_starting_points = [m_eye,h_eye]
    Gauss_quad_result = optimize(Gauss_quadparam -> Gauss_objective_quad(Gauss_quadparam, xcopy, ycopy, scopy),quad_starting_points);
    Gauss_lin_result = optimize(Gauss_linparam -> Gauss_objective_lin(Gauss_linparam, xcopy, ycopy, scopy),lin_starting_points);
    

#     @assert Optim.converged(Gauss_quad_result)
#     @assert Optim.converged(Gauss_lin_result)
    
    m_jack, h_jack = Optim.minimizer(Gauss_lin_result)
    M_jack[i] = m_jack
    H_jack[i] = h_jack
    
    a_jack, b_jack, c_jack = Optim.minimizer(Gauss_quad_result)
    A_jack[i] = a_jack
    B_jack[i] = b_jack
    C_jack[i] = c_jack
 
end

In [23]:
f = Figure()
Axis(f[1, 1])
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize=20, color=:red)

for i in 1:ndata
    yy_lin_jack  = xx .* M_jack[i] .+ H_jack[i]
    yy_quad_jack = A_jack[i].*xx.^2 .+ B_jack[i].*xx .+ C_jack[i]
    lines!(xx, yy_quad_jack, color=:blue)
    lines!(xx, yy_lin_jack, color=:grey)
end


gauss_yy_quad = a_gauss.*xx.^2 .+ b_gauss.*xx .+ c_gauss
lines!(xx, gauss_yy_quad, color=:red, linewidth=3)



gauss_yy_lin = m_gauss.*xx .+ h_gauss
lines!(xx, gauss_yy_lin, color=:black)


f

In the above plot, we can see the many different fits, each per removing one data point. The grey plots are for the linear jackknife and the blue one, are for quadratic jackknife method. In the following I am going to do the same process but this time for the __original data set__ without removing the outliers by hand. 

In [25]:
quad_starting_points = [a_eye,b_eye,c_eye]
lin_starting_points = [m_eye,h_eye]
Gauss_quad_result2 = optimize(Gauss_quadparam -> Gauss_objective_quad(Gauss_quadparam, dataset.x, dataset.y, dataset.sigma_y),quad_starting_points);
Gauss_lin_result2 = optimize(Gauss_linparam -> Gauss_objective_lin(Gauss_linparam, dataset.x, dataset.y, dataset.sigma_y),lin_starting_points);
a_gauss2, b_gauss2, c_gauss2 = Optim.minimizer(Gauss_quad_result2);
m_gauss2, h_gauss2 = Optim.minimizer(Gauss_lin_result2);




ndata = size(dataset,1)
# because we are going to have many runs, and each time removing one data point, it means that for 16 data points (npoints), we would get 16 different values for our fitting paramaters
A_jack2 = zeros(ndata) # quadratic fitting parameters 
B_jack2 = zeros(ndata)  # quadratic fitting parameters
C_jack2 = zeros(ndata)  # quadratic fitting parameters
M_jack2 = zeros(ndata)  # linear fitting parameters
H_jack2 = zeros(ndata)  # linear fitting parameters

#loop over the pulling points out cases 
for i in 1:ndata
    
    xcopy2 = copy(dataset.x)
    deleteat!(xcopy2, i)

    ycopy2 = copy(dataset.y)
    deleteat!(ycopy2, i)

    scopy2 = copy(dataset.sigma_y)
    deleteat!(scopy2, i)

    
    quad_starting_points = [a_eye,b_eye,c_eye]
    lin_starting_points = [m_eye,h_eye]
    Gauss_quad_result2 = optimize(Gauss_quadparam -> Gauss_objective_quad(Gauss_quadparam, xcopy2, ycopy2, scopy2),quad_starting_points);
    Gauss_lin_result2 = optimize(Gauss_linparam -> Gauss_objective_lin(Gauss_linparam, xcopy2, ycopy2, scopy2),lin_starting_points);
    

    @assert Optim.converged(Gauss_quad_result2)
    @assert Optim.converged(Gauss_lin_result2)
    
    m_jack2, h_jack2 = Optim.minimizer(Gauss_lin_result2)
    M_jack2[i] = m_jack2
    H_jack2[i] = h_jack2
    
    a_jack2, b_jack2, c_jack2 = Optim.minimizer(Gauss_quad_result2)
    A_jack2[i] = a_jack2
    B_jack2[i] = b_jack2
    C_jack2[i] = c_jack2
 
end

In [26]:
f = Figure()
Axis(f[1, 1])
errorbars!(dataset.x, dataset.y, dataset.sigma_y)
scatter!(dataset.x, dataset.y, markersize=20, color=:red)

for i in 1:ndata
    yy_lin_jack2  = xx .* M_jack2[i] .+ H_jack2[i]
    yy_quad_jack2 = A_jack2[i].*xx.^2 .+ B_jack2[i].*xx .+ C_jack2[i]
    lines!(xx, yy_quad_jack2, color=:blue)
    lines!(xx, yy_lin_jack2, color=:grey)
end


gauss_yy_quad2 = a_gauss2.*xx.^2 .+ b_gauss2.*xx .+ c_gauss2
lines!(xx, gauss_yy_quad2, color=:red, linewidth=3)



gauss_yy_lin2 = m_gauss2.*xx .+ h_gauss2
lines!(xx, gauss_yy_lin2, color=:black)


f

## IT IS A MESS :)
So, it is better to first remove the outliers by hand. then oduble check with the jackknife. 


Now, I am going to invistigate the scatter of A_jack, as is demanded:

In [27]:
plot the relative difference and the scatter of q
f = Figure()
Axis(f[1, 1])
scatter!(A_jack)
f

In [39]:
A_jack

16-element Vector{Float64}:
 0.0035785621612145083
 0.013180736842401303
 0.004565683217755459
 0.003406782345267763
 0.0033968378686399033
 0.005018161438303045
 0.0039060588239509426
 0.004328196194259242
 0.002307186144108033
 0.0037552738622059567
 0.004012545547356902
 0.0027276854833951482
 0.0036911010189031607
 0.004659532489310263
 0.0032720142109171563
 0.003769319192685607